# Klassifikation mit anderen Klassifikatoren

In diesem Teil wirst du kennenlernen, welche anderen Klassifikatoren es in `scikit-learn` noch gibt und wie sich diese bei der Vorhersage der Autoren verhalten.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

Alle diese Felder kommen damit prinzipiell als Kategorien in Frage. Betrachte sie nun nacheinander:

## Daten für Autoren-Klassifikation vorbereiten

Die Vorhersage der Autoren hat schon mit dem `SVC`-Modell sehr gut funktioniert. Deshalb wendest du dieses Beispiel nun auch auf die anderen Klassifikatoren an. Die Vorbereitung der Daten ist genau wie vorher:

Du bestimmst zunächst die Top-Autoren:

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

Damit du zu einer ausgeglichenen Trainingsmenge kommst, suchst du den Top-Autor mit den wenigsten Artikeln:

In [ ]:
min_articles = min(top_authors["title"])

Nun konstruierst du einen `DataFrame`, der von allen Autoren gleich viele Artikel enthält:

In [ ]:
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

Nun kannst du die Daten vektorisieren:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["nav"])
tfidf_vectors

## Allgemeine Klassifikation und Vorhersage

Um es dir etwas einfacher zu machen, die Vorhersagen der Modelle zu bewerten, schreibst du dir eine kleine Hilfsfunktion, ide den Klassifikator trainiert, die Vorhersage durchführt und die anzeigt, wie häufig die Vorhersage mit der Wahrheit übereinstimmt:

In [ ]:
import numpy as np

def train_and_predict(clf, X, y):
    clf.fit(X, y)
    predicted = clf.predict(X)
    print(np.unique(y == predicted, return_counts=True))

## Naive Bayes

Der *Naive Bayes*-Klassifikator ist sehr einfach, du kannst ihn extrem schnell trainieren und das Ergebnis ist einigermaßen ordentlich. *Naive Bayes* wird häufig als sog. *Baseline* verwendet - alles Anderes sollte eigentlich besser sein.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
train_and_predict(MultinomialNB(), tfidf_vectors, adf["author"].values)

## Logistic Regression

Im Gegensatz zu dem Namen, der nach Regression klingt, kannst du den *Logistic Regression* auch zur Klassifikation verwenden. Die Ergebnisse sind besser als bei Naive Bayer:

In [ ]:
from sklearn.linear_model import LogisticRegression
train_and_predict(LogisticRegression(), tfidf_vectors, adf["author"].values)

## Linear SVM

Der *SGDClassifier* ist eine SVM, die mit einem linearen Kernel arbeitet. Das Ergebnis ist fast zu gut und sieht nach "auswendig lernen" aus:

In [ ]:
from sklearn.linear_model import SGDClassifier
train_and_predict(SGDClassifier(loss="hinge"), tfidf_vectors, adf["author"].values)

## Decision Tree

*Decision Tree* baut eine Kaskade von ja/nein-Entscheidungen auf und versucht damit, aus den Daten zu lernen. Auch das sieht nach "auswendig" aus:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
train_and_predict(DecisionTreeClassifier(), tfidf_vectors, adf["author"].values)

## Random Forest

*Random Forest* ist eine sog. *Ensemble-Methode* aus vielen Decision Trees. Auch hier liegt der Verdacht nahe, dass das Modell nur auswendig gelernt hat:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
train_and_predict(RandomForestClassifier(), tfidf_vectors, adf["author"].values)

## Gradient Boost

*Gradient Boost*  ist eine Ensemble-Methode, bei der die Gewichtung der einzelnen Klassifikatoren dynamisch angepasst wird. Auch dieser Algorithmus hat vermutlich auswendig gelernt.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
train_and_predict(GradientBoostingClassifier(), tfidf_vectors, adf["author"].values)

## K nearest neighbors

*K nearest neighbors* klassifiziert einen unbekannten Punkt immer mit der Mehrheit (z.B. 5 bei k=5) der Punkte, zu denen die direkt umliegenden Punkte gehören. Der Algorithmus funktioniert in hochdimensionalen Räumen leider nicht sonderlich gut:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
train_and_predict(KNeighborsClassifier(), tfidf_vectors, adf["author"].values)

## Wahl des richtigen Klassifikators ist entscheidend

Wie du siehst, ist die Performance der unterschiedlichen Klassifikatoren sehr verschieden.

Noch besser bewerten kannst du das, wenn du die Performance-Metriken kennst.